In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('cleaning data_3.xlsx', converters={'Merchnum': lambda x: str(x), 'Merch_zip': lambda x : str(x)})

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96397 entries, 0 to 96396
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Recnum                  96397 non-null  int64         
 1   Cardnum                 96397 non-null  int64         
 2   Date                    96397 non-null  datetime64[ns]
 3   Merchnum                93199 non-null  object        
 4   Merch_description       96397 non-null  object        
 5   Merch_state             95464 non-null  object        
 6   Merch_zip               92097 non-null  object        
 7   Amount                  96397 non-null  float64       
 8   Fraud                   96397 non-null  int64         
 9   Merch_description_part  96397 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 7.4+ MB


## 1. Treat Merchnum == 0 as nan

In [4]:
df['Merchnum'] = df['Merchnum'].replace('0',np.nan)

In [5]:
df[df['Merchnum'] == '0']

,Recnum,Cardnum,Date,Merchnum,Merch_description,Merch_state,Merch_zip,Amount,Fraud,Merch_description_part


## 2. Filling in missing value

In [6]:
group_merchnum_1 = df.groupby('Merchnum')['Merch_zip'].apply(pd.Series.mode).to_frame().reset_index().groupby('Merchnum').agg({'Merch_zip':'first'})
group_merchnum_2 = df.groupby('Merchnum')['Merch_state'].apply(pd.Series.mode).to_frame().reset_index().groupby('Merchnum').agg({'Merch_state':'first'})

In [7]:
group_merchnum = df['Merchnum'].unique()
group_merchnum = pd.DataFrame({'Merchnum':group_merchnum})
group_merchnum = group_merchnum.set_index('Merchnum')

In [8]:
group_merchnum = pd.merge(group_merchnum, group_merchnum_1,left_index = True, right_index = True,how = 'left')
group_merchnum = pd.merge(group_merchnum, group_merchnum_2,left_index = True, right_index = True,how = 'left')

In [9]:
group_merchnum

,Merch_zip,Merch_state
Merchnum,,
5509006296254,38118,TN
61003026333,01803,MA
4503082993600,20706,MD
6098208200062,45429,OH
602608969534,30091,GA
...,...,...
30165377,60033,IL
936337,60123,IL
4503092553800,98188,WA


In [10]:
group_description_1 = df[~df['Merch_description'].str.contains('ADJ')].groupby('Merch_description')['Merch_zip'].apply(pd.Series.mode).to_frame().reset_index().groupby('Merch_description').agg({'Merch_zip':'first'})
group_description_2 = df[~df['Merch_description'].str.contains('ADJ')].groupby('Merch_description')['Merch_state'].apply(pd.Series.mode).to_frame().reset_index().groupby('Merch_description').agg({'Merch_state':'first'})
group_description_3 = df[~df['Merch_description'].str.contains('ADJ')].groupby('Merch_description')['Merchnum'].apply(pd.Series.mode).to_frame().reset_index().groupby('Merch_description').agg({'Merchnum':'first'})

In [11]:
group_description_4 = df[~df['Merch_description'].str.contains('ADJ')].groupby('Merch_description').agg({'Recnum':'first'})

In [12]:
group_description = df[~df['Merch_description'].str.contains('ADJ')]
group_description = group_description['Merch_description'].unique()
group_description = pd.DataFrame({'Merch_description':group_description})
group_description = group_description.set_index('Merch_description')

In [13]:
group_description = pd.merge(group_description,group_description_1,left_index = True, right_index = True, how = 'left')
group_description = pd.merge(group_description,group_description_2,left_index = True, right_index = True, how = 'left')
group_description = pd.merge(group_description,group_description_3,left_index = True, right_index = True, how = 'left')
group_description = pd.merge(group_description,group_description_4,left_index = True, right_index = True, how = 'left')

In [14]:
group_description['Merch_zip'] = group_description['Merch_zip'].fillna(group_description['Recnum'])
group_description['Merch_state'] = group_description['Merch_state'].fillna(group_description['Recnum'])

In [15]:
group_description

,Merch_zip,Merch_state,Merchnum,Recnum
Merch_description,,,,
FEDEX SHP 12/23/09 AB#,38118,TN,5509006296254,1
SERVICE MERCHANDISE #81,01803,MA,61003026333,2
OFFICE DEPOT #191,20706,MD,4503082993600,3
FEDEX SHP 12/28/09 AB#,38118,TN,5509006296254,4
FEDEX SHP 12/22/09 AB#,38118,TN,5509006296254,6
...,...,...,...,...
W G COMPUTER SPLY GRP,60033,IL,30165377,96679
THE BON MARCHE #8,98188,WA,4503092553800,96714
TEXACO 22601220094,36043,AL,580000003,96720


In [16]:
group_description.loc['AOAC INTERNATIONAL']

Merch_zip             20877
Merch_state              MD
Merchnum       253050503505
Recnum                10863
Name: AOAC INTERNATIONAL, dtype: object

In [17]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch_description,Merch_state,Merch_zip,Amount,Fraud,Merch_description_part
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,3.62,0,FEDEXSHPAB
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,31.42,0,SERVICEMERCHANDISE
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,178.49,0,OFFICEDEPOT
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,3.62,0,FEDEXSHPAB
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,3.62,0,FEDEXSHPAB


### 2.1 Fill state and zip by merchnum group

In [18]:
df = df.set_index('Merchnum')
df['Merch_state'] = df['Merch_state'].fillna(group_merchnum['Merch_state'])
df['Merch_state'] = df['Merch_state'].replace('nan', np.nan)
df['Merch_state'].count()

95464

In [19]:
df['Merch_zip'] = df['Merch_zip'].fillna(group_merchnum['Merch_zip'])
df['Merch_zip'] = df['Merch_zip'].replace('nan', np.nan)
df = df.reset_index()
df['Merch_zip'].count()

93923

### 2.1 Fill state, zip and merchnum by merch_description group

In [20]:
df = df.set_index('Merch_description')
df['Merch_state'] = df['Merch_state'].fillna(group_description['Merch_state'])
df['Merch_state'] = df['Merch_state'].replace('nan', np.nan)
df['Merch_state'].count()

95736

In [21]:
df['Merch_zip'] = df['Merch_zip'].fillna(group_description['Merch_zip'])
df['Merch_zip'] = df['Merch_zip'].replace('nan', np.nan)
df['Merch_zip'].count()

95701

In [22]:
df['Merchnum'] = df['Merchnum'].fillna(group_description['Merchnum'])
df['Merchnum'] = df['Merchnum'].replace('nan', np.nan)
df['Merchnum'].count()

94303

In [23]:
df = df.reset_index()

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96397 entries, 0 to 96396
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Merch_description       96397 non-null  object        
 1   Merchnum                94303 non-null  object        
 2   Recnum                  96397 non-null  int64         
 3   Cardnum                 96397 non-null  int64         
 4   Date                    96397 non-null  datetime64[ns]
 5   Merch_state             95736 non-null  object        
 6   Merch_zip               95701 non-null  object        
 7   Amount                  96397 non-null  float64       
 8   Fraud                   96397 non-null  int64         
 9   Merch_description_part  96397 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 7.4+ MB


### 2.3 Filling in missing zip and state using minus recordnum

In [25]:
df['minus'] = -df['Recnum']

In [26]:
df['minus'] = df['minus'].astype('str')

In [27]:
df['Merch_state'] = df['Merch_state'].fillna(df['minus'])
df['Merch_zip'] = df['Merch_zip'].fillna(df['minus'])

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96397 entries, 0 to 96396
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Merch_description       96397 non-null  object        
 1   Merchnum                94303 non-null  object        
 2   Recnum                  96397 non-null  int64         
 3   Cardnum                 96397 non-null  int64         
 4   Date                    96397 non-null  datetime64[ns]
 5   Merch_state             96397 non-null  object        
 6   Merch_zip               96397 non-null  object        
 7   Amount                  96397 non-null  float64       
 8   Fraud                   96397 non-null  int64         
 9   Merch_description_part  96397 non-null  object        
 10  minus                   96397 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 8.1+ MB


### 2.4 assign merchnum by grouping by merch_description & state & zip

In [29]:
df['Merch_state'] = df['Merch_state'].astype('str')
df['Merch_zip'] = df['Merch_zip'].astype('str')

In [30]:
df['assign'] = df['Merch_description'] + df['Merch_state'] + df['Merch_zip']

In [31]:
merchnum_missing = df.groupby('assign').agg({'minus':'first'})

In [32]:
df = df.set_index('assign')
df['Merchnum'] = df['Merchnum'].fillna(merchnum_missing['minus'])
df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96397 entries, 0 to 96396
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   assign                  96397 non-null  object        
 1   Merch_description       96397 non-null  object        
 2   Merchnum                96397 non-null  object        
 3   Recnum                  96397 non-null  int64         
 4   Cardnum                 96397 non-null  int64         
 5   Date                    96397 non-null  datetime64[ns]
 6   Merch_state             96397 non-null  object        
 7   Merch_zip               96397 non-null  object        
 8   Amount                  96397 non-null  float64       
 9   Fraud                   96397 non-null  int64         
 10  Merch_description_part  96397 non-null  object        
 11  minus                   96397 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), ob

In [33]:
df.head()

,assign,Merch_description,Merchnum,Recnum,Cardnum,Date,Merch_state,Merch_zip,Amount,Fraud,Merch_description_part,minus
0,FEDEX SHP 12/23/09 AB#TN38118,FEDEX SHP 12/23/09 AB#,5509006296254,1,5142190439,2010-01-01,TN,38118,3.62,0,FEDEXSHPAB,-1
1,SERVICE MERCHANDISE #81MA01803,SERVICE MERCHANDISE #81,61003026333,2,5142183973,2010-01-01,MA,01803,31.42,0,SERVICEMERCHANDISE,-2
2,OFFICE DEPOT #191MD20706,OFFICE DEPOT #191,4503082993600,3,5142131721,2010-01-01,MD,20706,178.49,0,OFFICEDEPOT,-3
3,FEDEX SHP 12/28/09 AB#TN38118,FEDEX SHP 12/28/09 AB#,5509006296254,4,5142148452,2010-01-01,TN,38118,3.62,0,FEDEXSHPAB,-4
4,FEDEX SHP 12/23/09 AB#TN38118,FEDEX SHP 12/23/09 AB#,5509006296254,5,5142190439,2010-01-01,TN,38118,3.62,0,FEDEXSHPAB,-5


In [34]:
df.to_excel('no_missing.xlsx')

In [35]:
df[df['Merch_description'] == 'AOAC INTERNATIONAL']

,assign,Merch_description,Merchnum,Recnum,Cardnum,Date,Merch_state,Merch_zip,Amount,Fraud,Merch_description_part,minus
10850,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,10863,5142268678,2010-02-15,MD,20877,929.5,0,AOACINTERNATIONAL,-10863
59263,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,59465,5142270247,2010-08-03,MD,20877,1225.0,0,AOACINTERNATIONAL,-59465
59682,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,59887,5142182005,2010-08-04,MD,20877,400.0,0,AOACINTERNATIONAL,-59887
60907,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,61120,5142270247,2010-08-09,MD,20877,975.0,0,AOACINTERNATIONAL,-61120
62164,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,62384,5142189945,2010-08-11,MD,20877,750.0,0,AOACINTERNATIONAL,-62384
63067,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,63288,5142270247,2010-08-14,MD,20877,975.0,0,AOACINTERNATIONAL,-63288
65286,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,65511,5142270247,2010-08-21,MD,20877,85.0,0,AOACINTERNATIONAL,-65511
71813,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,72089,5142274543,2010-09-11,MD,20877,475.0,0,AOACINTERNATIONAL,-72089
85113,AOAC INTERNATIONALMD20877,AOAC INTERNATIONAL,253050503505,85450,5142230181,2010-11-05,MD,20877,312.0,0,AOACINTERNATIONAL,-85450
85561,AOAC INTERNATIONALMD00002,AOAC INTERNATIONAL,253050503505,85903,5142189945,2010-11-08,MD,00002,660.0,0,AOACINTERNATIONAL,-85903


In [36]:
df[df['Merch_description'].str.contains('GRAINGER')].tail(20)

,assign,Merch_description,Merchnum,Recnum,Cardnum,Date,Merch_state,Merch_zip,Amount,Fraud,Merch_description_part,minus
90005,GRAINGER #935OH44056,GRAINGER #935,06-0006692427,90351,5142184598,2010-12-01,OH,44056,344.35,0,GRAINGER,-90351
90034,GRAINGER #937IL60188,GRAINGER #937,06-0004269242,90381,5142188959,2010-12-01,IL,60188,51.03,0,GRAINGER,-90381
90801,GRAINGER #236MI48104,GRAINGER #236,06-3666055741,91150,5142184598,2010-12-06,MI,48104,10.80,0,GRAINGER,-91150
92070,GRAINGER #330NC27834,GRAINGER #330,-22812,92420,5142211809,2010-12-10,NC,27834,80.55,0,GRAINGER,-92420
92266,GRAINGER #637NC27616,GRAINGER #637,06-3666057882,92616,5142211809,2010-12-10,NC,27616,323.46,0,GRAINGER,-92616
92587,GRAINGER #328NC27406,GRAINGER #328,-8165,92937,5142211809,2010-12-13,NC,27406,256.02,0,GRAINGER,-92937
92670,GRAINGER #637NC27616,GRAINGER #637,06-3666057882,93020,5142204042,2010-12-13,NC,27616,20.28,0,GRAINGER,-93020
92763,GRAINGER #933TX76010,GRAINGER #933,06-3666888433,93113,5142250016,2010-12-14,TX,76010,430.52,0,GRAINGER,-93113
93593,GRAINGER #732CA95131,GRAINGER #732,-4837,93943,5142277104,2010-12-16,CA,95131,876.20,0,GRAINGER,-93943
93882,GRAINGER #935OH44056,GRAINGER #935,06-0006692427,94232,5142185141,2010-12-17,OH,44056,132.64,0,GRAINGER,-94232


In [37]:
df[df['Merch_description'].str.contains('CELLULAR ONE')].tail(20)

,assign,Merch_description,Merchnum,Recnum,Cardnum,Date,Merch_state,Merch_zip,Amount,Fraud,Merch_description_part,minus
18204,CELLULAR ONECA94080,CELLULAR ONE,674678180330,18233,5142261097,2010-03-14,CA,94080,994.80,0,CELLULARONE,-18233
22953,CELLULAR ONEMI49686,CELLULAR ONE,9013200038298,23002,5142174719,2010-03-29,MI,49686,325.12,0,CELLULARONE,-23002
31708,CELLULAR ONE 10OR97330,CELLULAR ONE 10,08-3508354542,31795,5142234238,2010-05-01,OR,97330,359.98,0,CELLULARONE,-31795
34457,CELLULAR ONE 8MT59401,CELLULAR ONE 8,-34561,34561,5142189667,2010-05-10,MT,59401,59.99,0,CELLULARONE,-34561
34713,CELLULAR ONE #21CA94103,CELLULAR ONE #21,674088896331,34818,5142113833,2010-05-11,CA,94103,59.97,0,CELLULARONE,-34818
52209,CELLULAR ONE 9OR97365,CELLULAR ONE 9,08-3508354479,52389,5142235358,2010-07-12,OR,97365,29.00,0,CELLULARONE,-52389
54313,CELLULAR ONE 10OR97330,CELLULAR ONE 10,08-3508354542,54503,5142234238,2010-07-19,OR,97330,25.00,0,CELLULARONE,-54503
54360,CELLULAR ONE #290MT59601,CELLULAR ONE #290,08-0080002333,54550,5142189667,2010-07-19,MT,59601,110.00,0,CELLULARONE,-54550
65758,CELLULAR ONECA94080,CELLULAR ONE,674678180330,65984,5142261097,2010-08-22,CA,94080,345.99,0,CELLULARONE,-65984
67725,CELLULAR ONEFL32504,CELLULAR ONE,975096999222,67962,5142156165,2010-08-29,FL,32504,15.00,0,CELLULARONE,-67962
